In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('max_columns',100)

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
train = import_data('../input/pubg-finish-placement-prediction/train_V2.csv')
test = import_data('../input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
train.info()

In [ ]:
train.shape, test.shape

In [ ]:
y_train = train['winPlacePerc']
test_ids = test['Id']

train = train.drop(['Id', 'winPlacePerc'], axis=1)
test = test.drop('Id', axis=1)

In [ ]:
#dropping missing value
y_train.drop(2744604,inplace=True)
train.drop(2744604,inplace=True)

In [ ]:
data = pd.concat([train, test], axis=0).reset_index(drop=True)

In [ ]:
data['matchplayers']=data.groupby('matchId')['matchId'].transform('count')
data['traveldistance']=data['walkDistance']+data['swimDistance']+data['rideDistance']
data['healsnboosts']=data['heals']+data['boosts']
data['assist']=data['assists']+data['revives']

data.drop(columns=['walkDistance','swimDistance','rideDistance'],axis=1,inplace=True)
data.drop(columns=['heals','boosts'],axis=1,inplace=True)
data.drop(columns=['assists','revives'],axis=1,inplace=True)
data.drop(columns=['groupId','matchId'],axis=1,inplace=True)

data.head(5)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [9])], remainder='passthrough')
data = pd.DataFrame(ct.fit_transform(data))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(data)
data = pd.DataFrame(sc.transform(data))

In [ ]:
train = data.iloc[:4446965].reset_index(drop='True')
test = data.iloc[4446965:].reset_index(drop='True')

In [ ]:
from catboost import CatBoostRegressor
reg = CatBoostRegressor()
reg.fit(train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score

acc = -cross_val_score(reg,train,y_train,scoring='neg_mean_absolute_error', cv = 3)

In [ ]:
acc.mean()

In [ ]:
y_pred = reg.predict(test)

In [ ]:
submission = pd.DataFrame({'Id':test_ids,'winPlacePerc':y_pred})
submission.to_csv('submission.csv',index=False)
print(submission)